
<p><center>Ocean Protocol</p>
<p><center>Trilobite pre-release 0.1</center></p>
<img src="https://oceanprotocol.com/static/media/logo-white.7b65db16.png" alt="drawing" width="200" align="center"/>
</center>

# Test functionality of squid-py wrapper.

<img src="https://3c1703fe8d.site.internapcdn.net/newman/gfx/news/hires/2017/mismatchedey.jpg" alt="drawing" width="200" align="center"/>

## Section 1: Import modules, and setup logging

Imports

In [39]:
%%javascript
var msg="hi"
console.log(msg)

<IPython.core.display.Javascript object>

In [31]:
from pathlib import Path
import squid_py.ocean as ocean_wrapper
# from squid_py.utils.web3_helper import convert_to_bytes, convert_to_string, convert_to_text, Web3Helper
import sys
import random
import json
from pprint import pprint
import squid_py.ocean as ocean
import names
from squid_py.ddo import DDO

Logging

In [32]:
import logging
loggers_dict = logging.Logger.manager.loggerDict
logger = logging.getLogger()
logger.handlers = []
# Set level
logger.setLevel(logging.INFO)
FORMAT = "%(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"
DATE_FMT = "%Y-%m-%d %H:%M:%S"
formatter = logging.Formatter(FORMAT, DATE_FMT)
# Create handler and assign
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(formatter)
logger.handlers = [handler]
logger.info("Logging started")

20 - <ipython-input-32-f1bdb9cc4f0f> - <module>        - Logging started


## Section 2: Instantiate the Ocean Protocol interface

In [33]:
# The contract addresses are loaded from file
# CHOOSE YOUR CONFIGURATION HERE
# PATH_CONFIG = Path.cwd() / 'config_local.ini'
PATH_CONFIG = Path.cwd() / '..' / '..' / 'config_k8s_deployed.ini'
# PATH_CONFIG = Path.cwd() /  'config_local.ini'
# PATH_CONFIG = Path.cwd() / 'config_k8s.ini'
# PATH_CONFIG = Path.cwd() / 'config_local.ini'
assert PATH_CONFIG.exists(), "{} does not exist".format(PATH_CONFIG)

ocn = ocean.Ocean(PATH_CONFIG)
logging.info("Ocean smart contract node connected ".format())

20 - <ipython-input-33-82d55dcb12ef> - <module>        - Ocean smart contract node connected 


## Section 3: Users and accounts

List the accounts created in Ganache

In [34]:
ocn.update_accounts()
for address in ocn.accounts:
    print(ocn.accounts[address])

# These accounts have a positive ETH balance
for address, account in ocn.accounts.items():
    assert account.ether >= 0
    assert account.ocean >= 0

Account 0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1 with 0 Eth, 6 Ocean
Account 0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0 with 99998521380000000000 Eth, 36 Ocean
Account 0x22d491Bde2303f2f43325b2108D26f1eAbA1e32b with 0 Eth, 98 Ocean
Account 0xE11BA2b4D45Eaed5996Cd0823791E0C93114882d with 0 Eth, 94 Ocean
Account 0xd03ea8624C8C5987235048901fB614fDcA89b117 with 0 Eth, 59 Ocean
Account 0x95cED938F7991cd0dFcb48F0a06a40FA1aF46EBC with 0 Eth, 10 Ocean
Account 0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9 with 0 Eth, 66 Ocean
Account 0x28a8746e75304c0780E011BEd21C72cD78cd535E with 99998521380000000000 Eth, 57 Ocean
Account 0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E with 0 Eth, 99 Ocean
Account 0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e with 0 Eth, 999 Ocean


Get funds to users
By default, 10 wallet addresses are created in Ganache
A simple wrapper for each address is created to represent a user
Users are instantiated and listed

In [35]:
class User():
    def __init__(self, name, role, account_obj):
        self.name = name
        self.role = role
        self.account = account_obj

        logging.info(self)

    def __str__(self):
        try:
            ocean_token = self.account.ocean
        except:
            ocean_token = 0
        return "{:<20} {:<20} {} Ocean token".format(self.name, self.role, ocean_token)

users = list()
for i, acct_address in enumerate(ocn.accounts):
    if i%2 == 0: role = 'Data Scientist'
    else: role = 'Data Owner'
    user = User(names.get_full_name(), role, ocn.accounts[acct_address])
    users.append(user)

20 - <ipython-input-35-5cda20a83649> - __init__        - Robin French         Data Scientist       6 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Lucille Peoples      Data Owner           36 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Douglas Smith        Data Scientist       98 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Terry Heil           Data Owner           94 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Paul Mattingly       Data Scientist       59 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Alda Decarr          Data Owner           10 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Jonathan Kyseth      Data Scientist       66 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Fanny Loomis         Data Owner           57 Ocean token
20 - <ipython-input-35-5cda20a83649> - __init__        - Sarah Wong           Data Scient

List the users

In [22]:
for u in users: print(u)

Daniel Marriner      Data Scientist       6 Ocean token
Janet Goggin         Data Owner           36 Ocean token
Marion Lorusso       Data Scientist       98 Ocean token
Salvatore Jensen     Data Owner           94 Ocean token
Robert Rothrock      Data Scientist       59 Ocean token
Pamella Meredith     Data Owner           10 Ocean token
Patricia Graves      Data Scientist       66 Ocean token
Kam Davis            Data Owner           57 Ocean token
Anna Yurek           Data Scientist       99 Ocean token
Gary Heisler         Data Owner           999 Ocean token


Get some Ocean token

In [23]:
for usr in users:
    rcpt = usr.account.request_tokens(random.randint(0,100))
    ocn._web3.eth.waitForTransactionReceipt(rcpt)

for u in users: print(u)

20 - market          - request_tokens  - 0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1 requests 67 tokens, returning receipt


KeyboardInterrupt: 

## Section 4: Find and publish assets

An asset has a DDO, which in turn has Metadata

In [36]:
SAMPLE_DDO_PATH = Path.cwd()
SAMPLE_DDO_PATH

PosixPath('/home/batman/ocn/mantaray/jupyter_notebooks/1_blocks')

In [37]:
# Load a sample DDO
SAMPLE_DDO_PATH = Path.cwd() / '../../sample_assets' / 'ddo_sample_generated_1.json'
assert SAMPLE_DDO_PATH.exists()
with open(SAMPLE_DDO_PATH) as f:
    SAMPLE_DDO_JSON_DICT = json.load(f)

SAMPLE_DDO_JSON_STRING = json.dumps(SAMPLE_DDO_JSON_DICT)

this_ddo = DDO(json_text=SAMPLE_DDO_JSON_STRING)
assert this_ddo.validate()
service = this_ddo.get_service('Metadata')
values = service.get_values()
assert values['metadata']

In [38]:
this_ddo

In [30]:
this_ddo.validate()

True

In [ ]:

# The sample asset metadata is stored in a .json file
PATH_ASSET1 = Path.cwd() / 'sample_assets' / 'sample1.json'
assert PATH_ASSET1.exists()
with open(PATH_ASSET1) as f:
    dataset = json.load(f)

logging.info("Asset metadata for {}: type={}, price={}".format(dataset['base']['name'],dataset['base']['type'],dataset['base']['price']))

registered_asset = users[0].register_asset(dataset)

asset = ocn.metadata.register_asset(dataset)
assert ocean_provider.metadata.get_asset_ddo(asset['assetId'])['base']['name'] == asset['base']['name']
ocean_provider.metadata.retire_asset(asset['assetId'])

In [ ]:
asset_ddo = ocn.metadata.get_asset_ddo(dataset['assetId'])
assert ocn.metadata.get_asset_ddo(dataset['assetId'])['base']['name'] == dataset['base']['name']